In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install num2words
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

     |████████████████████████████████| 101 kB 5.4 MB/s 
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 708.0 MB 9.7 kB/s 
     |████████████████████████████████| 5.9 MB 38.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.6.0+cu101 which is incompatible.


In [ ]:
import zipfile
import librosa
from scipy.io.wavfile import read

import torch
from torch.nn import Module,Conv1d,BatchNorm1d,ReLU,Sigmoid,LSTMCell,Linear,ModuleList,Embedding,Sequential,Dropout,ReLU,Tanh,MSELoss,LSTM,GRU,AdaptiveAvgPool1d,CrossEntropyLoss
from torch.nn import functional as F
from torch.optim import Adam

import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from num2words import num2words
import re
import IPython.display as ipd
from torch.autograd import Variable


from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence



In [ ]:
url='/content/drive/MyDrive/Digest/LJSpeech-1.1.zip'
with zipfile.ZipFile(url) as zf:
       for member in tqdm(zf.infolist(), desc='Extracting '):
             zf.extract(member, './')
             

Extracting : 100%|██████████| 13102/13102 [01:06<00:00, 198.40it/s]


In [ ]:
def clean(text):
  x= re.sub(r"\([-^()]*\)", "",text)
  x=re.sub("""[^A-Za-z0-9]+""",' ',x)
  return x.lower()

In [ ]:
class char_tokenizer():
  def __init__(self,space=True):
    self.tokendict={chr(ord('a')+i):i+1 for i in range(26)}
    if(space):self.tokendict[' ']=27
  def __len__(self):
    return len(self.tokendict.keys)
  def tokenize(self,text):
    return [self.tokendict[i] for i in text]




In [ ]:
class tacotron_generator():
  def __init__(self,audio_path,text_path,config):
    self.apth=audio_path
    self.audio=os.listdir(audio_path)
    self.text=pd.read_csv(text_path)
    assert(len(self.audio)==len(self.text))
    self.mel_basis=librosa.filters.mel(config.sr, config.n_fft, n_mels=config.n_mel)
    self.config=config
    self.text_tokenizer=char_tokenizer()

    self.max_textlen=200
    self.max_spectlen=814
    assert((self.max_spectlen)%config.numframesperstep==0)
    self.step=config.numframesperstep



  def __len__(self):
     return len(self.audio)
    

  def aud2mel(self,path):


    # audio

    aud, _ = librosa.load(path,sr=self.config.sr)    # loading the audio librosa return by normalizing it
    stftspectrogram=np.abs(librosa.stft(y=aud, n_fft=self.config.n_fft, hop_length=self.config.frame_hop_ms, win_length=self.config.frame_length_ms))# parameters as per the paper  fft->default value  , abs to convert amplitude to magnitude 
    melspectrogram=torch.tensor(np.dot(self.mel_basis,stftspectrogram),dtype=torch.float32) 
    compressed=torch.log(torch.clamp(melspectrogram, min=1e-3) )   # learn  log dynamic range compression.

    return compressed



  def __getitem__(self,index):
    
    # loading from csv
    filename,text=self.text['filename'][index],self.text['text'][index]
    spectrogram=self.aud2mel(self.apth+filename+'.wav')
    spectrogram=spectrogram.transpose(0,1) # since  rnn model shape must be (seqlen,dimension)  but librosa gave as(n_mel,seqlen)


    # cleaning text
    cleaned_sentence = re.findall(r'[A-Za-z]+|\d+', clean(text))
   
    cleanedtext=' '.join([num2words(i).replace(',','').replace('-',' ') if i.isnumeric() else i for i in cleaned_sentence])
    tokenized_text=self.text_tokenizer.tokenize(cleanedtext)

    ln=len(tokenized_text)
    pad=self.max_textlen-ln
    tokenized_text.extend([0]*pad)

    mask=np.zeros(self.max_textlen)
    mask[:ln]=1



    
    




    # padding spectrogram
    padspec=self.max_spectlen-spectrogram.shape[0]
    padzeros=torch.zeros(padspec,self.config.n_mel)

    op_mask=[1]*spectrogram.shape[0]+[0]*padspec
    oplen=spectrogram.shape[0]

    stop_token=[0]*self.max_spectlen
    stop_token[oplen-1]=1
    

    spectrogram=torch.cat([spectrogram,padzeros],dim=0)


    
    

    if(self.step>1):
        a,b=spectrogram.shape
        spectrogram=spectrogram.view(a//self.step,self.step,b)
        spectrogram=spectrogram.view(a//self.step,b*self.step)

    #inv_mel_basis = np.linalg.pinv(self.mel_basis)
    """
    for i in spectrogram:
      spectrogram = np.dot(inv_mel_basis, torch.exp(i.view(-1,80).transpose(0,1)).numpy())
      y = librosa.griffinlim(spectrogram)
      ipd.display(ipd.Audio(y, rate=22050))
    print(i) """
    # padding text 

    

    return  {'text':torch.tensor(tokenized_text),
             'op':spectrogram,
             'len':torch.tensor(ln,dtype=torch.int32),
             'mask':torch.tensor(mask,dtype=torch.int32),
             'opmask':torch.tensor(op_mask,dtype=torch.float32),
             'opl':torch.tensor(oplen),
             'stoptoken':torch.tensor(stop_token,dtype=torch.float32)
             }



     




In [ ]:
class ENCODER(Module):
  def __init__(self,config):
    super().__init__()

    self.embedding=Embedding(config.num_tokens, config.enc_dim)
    self.rnn= GRU(config.enc_dim, config.enc_dim//2,bidirectional=True,batch_first=True)
    self.fc1=Linear(config.enc_dim*2 ,config.enc_dim)
  
  def forward(self,X,lens):


    y=self.embedding(X)


    y = pack_padded_sequence(y,lens , batch_first=True,enforce_sorted=False)
    self.rnn.flatten_parameters()
    output, state = self.rnn(y)
    output, _ = pad_packed_sequence(output, batch_first=True, padding_value=0.)

    return output, state



In [ ]:
class Location_awarness(Module):
  def __init__(self,config):
    super().__init__()

    padding = int((config.attn_kernel - 1) / 2)
    self.Conv = Conv1d(2, config.attn_filter,config.attn_kernel ,padding=padding,bias=False)

    # fully connected layer to convert the features extracted from [cummulative,prev] for encoderop len  to features of attention dimension
    self.fc   = Linear(config.attn_filter,config.attn_dim,bias=False)

  def forward(self,stacked_probs):

    y=self.Conv(stacked_probs)
    y=torch.transpose(y,1,2)
    y=self.fc(y)

    return y


class Attention(Module):
  def __init__(self,config):
    super().__init__()

    self.W1=Linear(config.enc_dim,config.attn_dim,bias=False)
    self.W2=Linear(config.dec_dim,config.attn_dim,bias=False)
    self.V =Linear(config.attn_dim,1,bias=False)

    self.location=Location_awarness(config)

  def forward(self,prev_hidden,encoder_op,prev_attn,mask):


    eo=self.W1(encoder_op)
    ph=self.W2(prev_hidden.unsqueeze(1))

    pa=self.location(prev_attn)

    scores=self.V(torch.tanh(eo+ph+pa))


    scores.data.masked_fill_(mask.unsqueeze(-1) == 0, -1e10)
    #print(scores.detach().tolist())
    #print((mask.unsqueeze(-1) == 0).detach().tolist())

    attn_probs= F.softmax(scores, dim = 1)

    context_vector = attn_probs * encoder_op
    context_vector = torch.sum(context_vector, dim=1)

    return context_vector,attn_probs.squeeze(-1)

  

In [ ]:
class PRENET(Module):
    def __init__(self, config):
        super().__init__()
        self.layers = ModuleList(
            [Linear(config.spectrogram_dimension,config.prenet_dim,bias=False),
             Linear(config.prenet_dim,config.prenet_dim,bias=False)]
            )

    def forward(self, x):
        for linear in self.layers:
            x = F.dropout(F.relu(linear(x)), p=0.5, training=True)
        return x


In [ ]:
class DECODER(Module):
  def __init__(self,config):
    super().__init__()

    self.prenet=PRENET(config)
    self.rnn1 = GRU(config.prenet_dim + config.contextvec_dim, config.dec_dim,batch_first=True)
    self.rnn2 = GRU(config.dec_dim + config.contextvec_dim, config.dec_dim,batch_first=True)
    self.attention=Attention(config)

    self.spec_pred=Linear(config.dec_dim+config.contextvec_dim,config.spectrogram_dimension,bias=False)
    self.stop_pred=Linear(config.dec_dim+config.contextvec_dim,1,bias=False) # stop is binary

    self.bridge=Linear(config.enc_dim,config.dec_dim,bias=False)

    self.drop1=Dropout(0.1)
    self.drop2=Dropout(0.1)
    

    self.nst=config.spectrogram_dimension

    

  def compute_step(self,curr_ip,prev_hidden,encoder_op,mask):

    prev_attn=torch.stack([self.prev_attn_probs,self.prev_Atn_cum],dim=1)
    context_vector,self.prev_attn_probs=self.attention(prev_hidden,encoder_op,prev_attn,mask)
    self.prev_Atn_cum=self.prev_Atn_cum+self.prev_attn_probs


    y=self.prenet(curr_ip)


    y=torch.cat([y,context_vector],dim=-1)
    y=torch.unsqueeze(y,1) # making seq of len 1
    y,_=self.rnn1(y)
    y=self.drop1(y)
    
    y=torch.cat([y,context_vector.unsqueeze(1)],dim=-1)
    y,state=self.rnn2(y,prev_hidden.unsqueeze(0))
    y=self.drop2(y)

    y=torch.squeeze(y,1) # making seq of len 1 to a vector of tht time step

    y=torch.cat([y,context_vector],dim=-1)

    spec=self.spec_pred(y)
    stop=self.stop_pred(y)

    return spec,state,stop.squeeze(-1)


  def start_token(self,B):
    return  Variable(torch.zeros(B,self.nst))

  def forward(self,encoder_ops,decoder_ip,oldencoder_mask):

    max_len=decoder_ip.shape[1]
    encoder_op,state=encoder_ops
    B=encoder_op.shape[0]
    curr_ip=self.start_token(B).to(encoder_op.device)
    hidden=self.bridge(torch.cat([state[0],state[1]],-1))

    spectrogram_op=[]
    stop_token_op=[]

    encoder_mask=oldencoder_mask[:,:encoder_op.shape[1]]


    self.prev_Atn_cum=Variable(encoder_op.data.new(B, encoder_op.shape[1]).zero_())
    self.prev_attn_probs=Variable(encoder_op.data.new(B, encoder_op.shape[1]).zero_()) 

    for t in range(max_len):




          predictions, hidden,stop_pred = self.compute_step(curr_ip,hidden,encoder_op,encoder_mask)



          spectrogram_op.append(predictions)
          stop_token_op.append(torch.sigmoid(stop_pred))
          curr_ip = decoder_ip[:, t]
          #if(t in [0,1,2,5,10,20,50,100,200,500]):print(F.mse_loss(predictions,curr_ip).item(),t)

          hidden=hidden.squeeze(0)

          

    spectrogram_op=torch.stack(spectrogram_op).transpose(0,1)
    stop_token_op=torch.stack(stop_token_op).transpose(0,1)
    return spectrogram_op,stop_token_op

  def inference(self,encoder_ops,oldencoder_mask):

    max_len=814
    encoder_op,state=encoder_ops
    B=encoder_op.shape[0]
    curr_ip=self.start_token(B).to(encoder_op.device)
    hidden=self.bridge(torch.cat([state[0],state[1]],-1))

    spectrogram_op=[]
    stop_token_op=[]


    encoder_mask=oldencoder_mask[:,:encoder_op.shape[1]]

    self.prev_Atn_cum=Variable(encoder_op.data.new(B, encoder_op.shape[1]).zero_())
    self.prev_attn_probs=Variable(encoder_op.data.new(B, encoder_op.shape[1]).zero_()) 
    #print()
    for t in range(max_len):

          predictions, hidden,stop_pred = self.compute_step(curr_ip,hidden,encoder_op,encoder_mask)
          spectrogram_op.append(predictions)
          stop_token_op.append(torch.sigmoid(stop_pred))

          curr_ip = predictions

          hidden=hidden.squeeze(0)

    spectrogram_op=torch.stack(spectrogram_op).transpose(0,1)
    return spectrogram_op,stop_token_op






In [ ]:
class Resnet_block(Module):
  def __init__(self,indim,outdim,kernel):

    super().__init__()

    self.conv1 = Conv1d(indim,outdim,kernel,padding=(kernel-1)//2,bias=False)
    self.bn1   = BatchNorm1d(outdim)

    self.conv2 = Conv1d(outdim,outdim,kernel,padding=(kernel-1)//2,bias=False)
    self.bn2   = BatchNorm1d(outdim)

    self.skip=Sequential(
        Conv1d(indim,outdim,1,bias=False),
        BatchNorm1d(outdim)
    )

  def forward(self,X):

    y= self.bn1(self.conv1(X))
    y= self.bn2(self.conv2(y))  +self.skip(X)

    return torch.tanh(y)

class Enhancer(Module):
  def __init__(self,config):
    super().__init__()

    self.blocks=ModuleList()
    self.fc=ModuleList()

    num_layers=4

    for i in range(num_layers):

      ind=config.spectrogram_dimension if i==0 else config.enchancer_dim
      outd=config.enchancer_dim if i< num_layers-1 else config.spectrogram_dimension

      self.blocks.append(Resnet_block(ind,outd,5))
      self.fc.append(Linear(outd,outd,bias=False))

    self.avgpool = AdaptiveAvgPool1d(1)
    self.N=num_layers
    self.loss=MSELoss()

  def forward(self,X,training=True):

    y=X.transpose(1,2)



    for i in range(self.N):

      y=F.dropout(self.blocks[i](y),0.25)
      """
      z=self.avgpool(y)
      
      z=torch.squeeze(z,-1)
      z=self.fc[i](z)
      z=torch.sigmoid(z)
      z=torch.unsqueeze(z,-1)
      # channel attention did not have good result
      y=y*z+z """
      
    return y.transpose(1,2)

In [ ]:
class EDAttn(Module):
  def __init__(self,config):
    super().__init__()
    self.encoder=ENCODER(config)
    self.decoder=DECODER(config)
    self.enhancer=Enhancer(config)
    self.loss=MSELoss(reduction='none')
    self.lossstop=MSELoss()

  def loss_spectrogram(self,prediction,target,mask) : #summed mean squared error (MSE) from before and after the post-net to aid convergence
       
       a,b,stop=prediction
       spec_target,stop_target=target

       stop_loss=self.lossstop(stop,mask)
     
       mask=mask.unsqueeze(-1).expand_as(a)


       aloss=(self.loss(a,spec_target)* mask.float()).sum()/  mask.sum()
       bloss=(self.loss(b,spec_target)* mask.float()).sum()/  mask.sum() 


    
       return [aloss,bloss,stop_loss]
         
 
  def forward(self,input,decoder_input,mask,lens):

    y=self.encoder(input,lens)

    spec,stop=self.decoder(y,decoder_input,mask)
    spech_enh=self.enhancer(spec)+spec

    return spec,spech_enh,stop

  def inference(self,input,mask,lens):
    y=self.encoder(input,lens)
    y,stop=self.decoder.inference(y,mask)
    y1=self.enhancer(y)+y
    return y,y1,stop



  

In [ ]:
class fftconfig():
  def __init__(self,sample_rate):
    self.n_fft=2048

    
    self.sr=sample_rate
    self.frame_hop_ms=int(12.5 /1000 *  sample_rate)
    self.frame_length_ms=int(50  /1000 * sample_rate)
    self.n_mel= 80
    self.aud_max_val=32768.0
    self.numframesperstep=1

x=tacotron_generator("/content/LJSpeech-1.1/wavs/","/content/LJSpeech-1.1/dataset.csv",fftconfig(22050))
a=x[5]

In [ ]:

def avg(x):
  return sum(x)/len(x)
class pipeline():

    def __init__(self,model,processing_unit):
        
     self.device = torch.device(processing_unit)
     self.model=model
     self.model.to(self.device)
    
    def set_optimizer(self,optimizer,lr=0.0001):
     if(optimizer=='Adam'):
      self.optim = Adam(self.model.parameters(), lr=lr,weight_decay=1e-5)# weight decay tts paper
    def loadmodel(self,path):
      self.model.load_state_dict(torch.load(path,map_location=self.device))  


    def predict(self,text,vocoder=None):
     
     self.model.eval()
     cleaned_sentence = re.findall(r'[A-Za-z]+|\d+', clean(text))   
     cleanedtext=' '.join([num2words(i).replace(',','').replace('-',' ') if i.isnumeric() else i for i in cleaned_sentence])
     tokenized_text=char_tokenizer().tokenize(cleanedtext)

     ln=len(tokenized_text)
     mask=np.ones(ln)
     data={'text':torch.tensor(tokenized_text),
             'len':torch.tensor(ln,dtype=torch.int32),
             'mask':torch.tensor(mask,dtype=torch.int32),}



     for key,value in data.items():
              data[key]=value.unsqueeze(0)

     inp,mask,enc_len=data['text'],data['mask'],data['len']

     with torch.no_grad():
         spec_op,spec_enh,stop=self.model.inference(inp,mask,enc_len)                       
    

    
     self.mel_basis=librosa.filters.mel(22050, 2048, n_mels=80)
     inv_mel_basis = np.linalg.pinv(self.mel_basis)


     spectrogram = np.dot(inv_mel_basis, torch.exp(spec_op[0].transpose(0,1)).numpy())
     y = librosa.griffinlim(spectrogram)
     ipd.display(ipd.Audio(y*8, rate=22050))

     spectrogram = np.dot(inv_mel_basis, torch.exp(spec_enh[0].transpose(0,1)).numpy())
     y = librosa.griffinlim(spectrogram)
     ipd.display(ipd.Audio(y*8, rate=22050))

     if(vocoder):
       with torch.no_grad():
            print(spec_enh.transpose(1,2).shape)
            audio = vocoder.inference(spec_enh.transpose(1,2))
            ipd.display(ipd.Audio(audio*2, rate=22050))


     

     #print(stop)


   
    def inference(self,data):
     
     self.model.eval()

     for key,value in data.items():
              data[key]=value.unsqueeze(0)

     inp,spec_tar,mask,enc_len=data['text'],data['op'],data['mask'],data['len']
     



     with torch.no_grad():
         spec_op1,spec_op2,stop_op=self.model(inp,spec_tar,mask,enc_len)        
         losses=self.model.loss_spectrogram([spec_op1,spec_op2,stop_op],[spec_tar,data["stoptoken"]],data["opmask"])#+self.model.loss_stop_token(stop_op,stop,mask)
         loss=sum(losses)   
                  
    

     print(loss)
     self.mel_basis=librosa.filters.mel(22050, 2048, n_mels=80)
     inv_mel_basis = np.linalg.pinv(self.mel_basis)


     spectrogram = np.dot(inv_mel_basis, torch.exp(spec_op1[0].transpose(0,1)).numpy())
     y = librosa.griffinlim(spectrogram)
     ipd.display(ipd.Audio(y, rate=22050))

     spectrogram = np.dot(inv_mel_basis, torch.exp(spec_op2[0].transpose(0,1)).numpy())
     y = librosa.griffinlim(spectrogram)
     ipd.display(ipd.Audio(y, rate=22050))

     #print(stop_op)
     del loss       
                       
                     

                       


              
    def fit(self,data_generator,epochs,batch_size,save_best=False):
     self.generator= data_generator
     train_data_loader = DataLoader(data_generator, batch_size=batch_size,shuffle=True, num_workers=1)
     best_loss = float('inf')
     self.model.train()
     for epoch in range(epochs):
                 
                 final_loss=0
                 checkloss=[]
                
                 t=tqdm(train_data_loader,total=len(train_data_loader))
                 
                 for data in t:
                       for key,value in data.items():
                             data[key]=value.to(self.device)


                       self.optim.zero_grad()
                       
                       inp,spec_tar,mask,enc_len=data['text'],data['op'],data['mask'],data['len']
                       spec_op1,spec_op2,stop_op=self.model(inp,spec_tar,mask,enc_len)                       
                       losses=self.model.loss_spectrogram([spec_op1,spec_op2,stop_op],[spec_tar,data["stoptoken"]],data["opmask"])#+self.model.loss_stop_token(stop_op,stop,mask)
                       loss=sum(losses)
                       
                       loss.backward()
                       self.optim.step()

                       checkloss.append(loss.item() )

                       string="epoch  "+str(epoch)+"   loss: {:.8f}".format(avg(checkloss)) + ' '.join(["  {:.4f}  ".format(ls) for ls in losses])
                       t.set_description(string)
                       t.refresh() 

                       final_loss+=loss.item() 


                 save=''
                 if final_loss < best_loss:
                   if(save_best):
                           torch.save(self.model.state_dict(),save_best)
                   best_loss = final_loss
                   save="   saving model"
                 print("epoch :  ",epoch,"loss :  ",final_loss/len(train_data_loader),save)
    


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
class fftconfig():
  def __init__(self,sample_rate):
    self.n_fft=2048 # default

    
    self.sr=sample_rate
    self.frame_hop_ms=int(12.5 /1000 *  sample_rate) #paper  
    self.frame_length_ms=int(50  /1000 * sample_rate) #paper
    self.n_mel= 80
    self.aud_max_val=32768.0
    self.numframesperstep=1

train=tacotron_generator("/content/LJSpeech-1.1/wavs/","/content/LJSpeech-1.1/dataset.csv",fftconfig(22050))

In [ ]:
class cfg():
  def __init__(self,no_tokens):

    #default
    self.enc_dim=512 # conventional encoder
    self.dec_dim=1024 # 1024 worked better than 768
    self.attn_dim=128  # 256 cuda overflow

    self.contextvec_dim=self.enc_dim # adding encoder op on timesteps weighted scores will result vecotor of encder dimension

    #encoder
    self.num_tokens=no_tokens
  

    # attn location awarness (same as location awareness paper)
    self.attn_filter=32
    self.attn_kernel=31

    #spectrogram
    self.spectrogram_dimension=80 # number of mel channels
    self.no_framesperstep=1
    self.spectrogram_dimension*=self.no_framesperstep
    # number of frames to be predicted is 1  because it gives good results
    
    # prenet 
    self.prenet_dim=64 * self.no_framesperstep
    # checking 64 ,128,256      64 didn't have overfitting while others had

    # postnet
    self.enchancer_dim=256

In [ ]:



component=EDAttn(cfg(28))


Model=pipeline(component,'cpu')
Model.loadmodel("/content/drive/MyDrive/Digest/text2spectrebuild.pth")


Model.set_optimizer('Adam',lr= 1e-4 ) # 5e-4 worked best
#Model.fit(train,200,64,"/content/drive/MyDrive/Digest/text2spectrebuild.pth")
#Model.inference(train[105])
#Model.predict("kamalesh vignesh mitran sudhan siva aadhavan dhayanadh nitin harsha digest audio casette kamalesh vignesh mitran sudhan siva aadhavan dhayanadh nitin harsha digest audio casette kamalesh vignesh mitran sudhan siva aadhavan dhayanadh nitin harsha digest audio casette")
# text2spectantest2 -> text2spectantest21
#768 to 1024

In [ ]:
vocoder = torch.hub.load('seungwonpark/melgan' , 'melgan')
vocoder.eval()

Using cache found in /root/.cache/torch/hub/seungwonpark_melgan_master


In [ ]:
#Model.inference(train[105])
Model.predict("kamalesh vignesh mitran nitin harsha digest audio casette we are the boys",vocoder)

torch.Size([1, 80, 814])


In [ ]:
Model.set_optimizer('Adam',lr=5e-4)
Model.fit(train,200,64,"/content/drive/MyDrive/Digest/text2spectantest.pth")

  0%|          | 0/205 [01:56<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
observed 
250 ts ->0.3 stuck